In [214]:
import finite_differences.finite_diff_formulas as f
import importlib as i
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
i.reload(f)
import test_fd_methods as t
i.reload(t)
import test_implementation as ti
i.reload(ti)

<module 'test_implementation' from '/Users/manuelpanora/Python/Finite-Difference-Option-Pricer/test_implementation.py'>

Goal: Price a European call option. It is currently worth $100 strike price is $95 and it expires in 21 days. The risk free interest rate is 5% and the volatility is .2.  

In [215]:
TRADING_DAYS = 252
K = 95
S = 100
sigma = .2
r = .05
tfinal = 21 / TRADING_DAYS          

bs_price = f.black_scholes_call(S, K, tfinal, r, sigma)
print("European Call Option Price BS:", bs_price)

European Call Option Price BS: 5.898921658718436


This is the price computed given the initialized variables.

In [216]:
option = f.call_option(K)
#n is timesteps, amount discretize time
n = 16
#m is pricesteps, amount discretize price
m = 16
price_fd = f.finite_difference_call(n, m, K, tfinal, S, sigma, r)
print("European Call Option Price FD:", price_fd)

European Call Option Price FD: 5.876594250948936


The results show that with just 4 timesteps and 4 pricesteps, we can get very close to the actual values. We should compute the error and see how we can reduce it.
So far our results show that:
European Call Option Price BS: 5.898921658718436
European Call Option Price FD: 5.566651347113716

With 8 timesteps and 8 pricesteps we get 3 digits of accuracy, sufficient for daily trades:
European Call Option Price BS: 5.898921658718436
European Call Option Price FD: 5.893540752387366

For the next part, the crank nicolson method is implemented to price a European Call Option. We begin with the construction of the grid. First we will introduce the following substitutions: \tau = Tfinal - t, x = ln(S)

We introduce these substitutions so that we can transform the BS formula into the heat equation and then solve an ODE problem. We also want to work by moving forwawrd in time rather than backwards which is why we introduce \tau.
We get V(S,\tau) = e^{-r\tau}e^{ax+b\tau}w(x,\tau) and we solve the following equation: \partial{V}/\partial{\tau} = \sigma^2/2\partial^2{V}/\partial{x}^2
To allow for the transformation we must introduce the constants a and b, defined below

In [217]:
a = .5 - r/(sigma**2)
b =  -(a * sigma) ** 2 / 2 - r * a

To move forward with CN method, we must define the grid with the boundary conditions.

In [218]:
price_cn = f.price_derivative_cn(n, m, K, r, tfinal, a, b, S, sigma)

We have completed the creation of the Crank Nicolson Implentation. It is compared to the previous methods in the following section:

In [219]:
print("When given", m, "pricesteps and", n, "timesteps:")
print("European Call Option Price BS:", bs_price)
print("European Call Option Price FD:", price_fd)
print("European Call Option Price CN:", price_cn)

When given 16 pricesteps and 16 timesteps:
European Call Option Price BS: 5.898921658718436
European Call Option Price FD: 5.876594250948936
European Call Option Price CN: 4.64658489587504


From here, we are going to test and see which variable can be optimized best to see how we can improve it.

In [220]:
cn_errors, fd_errors, cn_values = t.test_convergence_all(K, r, tfinal, a, b, S, sigma, (5,6))
print(cn_errors)
print(fd_errors)

8.939974858729533
7.665218120566058
5.513540914986314
5.420673959363125
5.402685267568507
5.402230820988529
8.939974664408169
7.665215748657715
5.513532377111029
5.422342352919612
5.4056227285358025
5.405232969280241
8.939974606831594
7.665215046097496
5.513529843204733
5.422821020457848
5.406261879175253
5.405892581030524
8.939974600169705
7.665214964816263
5.513529549902814
5.422875968996582
5.406333113320031
5.40596396666049
8.939974599670014
7.665214958711927
5.5135295278754395
5.422880091963266
5.406338457458617
5.4059693230895745
[[21.53603729  2.46301591  0.25580463  0.54932452  1.40406453  3.76092605]
 [21.58442458  2.37518069  0.3911387   0.43488538  1.50428362  4.15038677]
 [21.61661958  2.31670348  0.48008758  0.36409983  1.58099805  4.37771809]
 [21.62831435  2.29545488  0.51218522  0.33923568  1.6096759   4.45437638]
 [21.63157682  2.28952654  0.5211194   0.3323742   1.61773296  4.47522309]]
[[3.0410532  1.76629646 0.38538074 0.4782477  0.49623639 0.49669084]
 [3.04105301 

As the time steps increase, the error between the actual value and the computed value are pretty bad. 